# 🔥Causal Graph Neural Networks for Wildfire Danger Prediction🔥
Re-implementation of original work by Zhao et al.(2024) (https://arxiv.org/abs/2403.08414)

IDL S25 Group 23: Wenting Yue, Wenyu Liu, Youyou Huang (Group 23)

## Retrieve files from github repository
If `only notebook` is downloaded locally

In [1]:
import os
import sys

# Get the current working directory
print("Current working directory:", os.getcwd())
# repo = "https://github.com/youyouh511/11785_IDL_S25_Final-Project.git"
# !git clone {repo}
!git pull

Current working directory: /home/ubuntu/11785_IDL_S25_Final-Project
Already up to date.


# Set up

## Environment

Environment setup
```bash
conda env create -f env.yml
```

Activate environment and check device
```bash
conda activate idl
nvidia-smi
python -c "import torch; print('CUDA available:', torch.cuda.is_available())"
```

# Imports

In [2]:
from data import (
    JsonFireDataset
)
from model import (
    AdjacencyMatrix,
    TemporalLSTM,
    CausalGNN
)
# from train import (
    
# )
# from utils import (
# )


import numpy as np
import tqdm
import matplotlib.pyplot as plt
import json
import zipfile
import torch
import requests
import xarray as xr
import yaml
from torchinfo import summary
import shutil
import wandb
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


# Config

In [3]:
%%writefile config.yaml

###### Dataset
    root                    : "./data"
    train_json_path         : "train.json"
    val_json_path           : "val.json"
    test_json_path          : "test.json"
    matrix_json_path        : "matrix.json"
    subset                  : 1.0
    batch_size              : 128
    NUM_WORKERS             : 4

    ### Target threshold
    fire_threshold          : 10


###### Model
    ### Adjacency matrix
    local_var_lag           : 8
    oci_var_lag             : 31
    max_lag                 : 312
    independence_test       : "ParCorr"
    tau_max                 : 186
    pc_alpha                : 0.05
    mask_target             : True

    ### Temporal LSTM
    lstm_layer              : 1
    hidden_dim              : 256

    ### GNN
    gnn_nodes               : 7


###### Training
    epochs                  : 30

    lr                      : 1.0e-5
    min_lr                  : 1.0e-9
    
    optimizer               : "Adam"
    betas                   : [0.9, 0.999]
    eps                     : 1.0e-8
    weight_decay            : 5.0e-6

    lr_scheduler            : "CosineAnnealingLR"
    patience                : 10
    early_stop              : True

    save_model              : True
    save_model_path         : "./checkpoints"
    load_model              : False
    load_model_path         : "./checkpoints/best.pth"
    wandb_log               : True
    wandb_project           : "IDL_Final"
    wandb_run_id            : None

Overwriting config.yaml


In [4]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config

{'root': './data',
 'train_json_path': 'train.json',
 'val_json_path': 'val.json',
 'test_json_path': 'test.json',
 'matrix_json_path': 'matrix.json',
 'subset': 1.0,
 'batch_size': 128,
 'NUM_WORKERS': 4,
 'fire_threshold': 10,
 'local_var_lag': 8,
 'oci_var_lag': 31,
 'max_lag': 312,
 'independence_test': 'ParCorr',
 'tau_max': 186,
 'pc_alpha': 0.05,
 'mask_target': True,
 'lstm_layer': 1,
 'hidden_dim': 256,
 'gnn_nodes': 7,
 'epochs': 30,
 'lr': 1e-05,
 'min_lr': 1e-09,
 'optimizer': 'Adam',
 'betas': [0.9, 0.999],
 'eps': 1e-08,
 'weight_decay': 5e-06,
 'lr_scheduler': 'CosineAnnealingLR',
 'patience': 10,
 'early_stop': True,
 'save_model': True,
 'save_model_path': './checkpoints',
 'load_model': False,
 'load_model_path': './checkpoints/best.pth',
 'wandb_log': True,
 'wandb_project': 'IDL_Final',
 'wandb_run_id': 'None'}

# Data Retrieval & Pre-process
Refer to data_preprocessing.ipynb

# Datasets

In [5]:
local_keys  = ["T2M_MEAN","TP","VPD"]
oci_keys    = ["OCI_NAO", "OCI_NINA34_ANOM", "OCI_AO"]

train_ds = JsonFireDataset(
    json_path   = "data/train.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)
val_ds = JsonFireDataset(
    json_path   = "data/val.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)
test_ds = JsonFireDataset(
    json_path   = "data/test.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)

# Model

## Adjacency Matrix

In [ ]:
matrix_builder = AdjacencyMatrix("data/matrix.json", independence_test="GPDCtorch",tau_max=23)
norm_matrix = matrix_builder.normalize_adj_matrix(matrix_builder.adj_matrix)
mask_matrix = matrix_builder.mask_target(norm_matrix, config['mask_target'])
print(matrix_builder.varlist)


### Panel 0 last 5 rows ###
     OCI_AO  OCI_NAO  OCI_NINA34_ANOM    T2M_MEAN         TP       VPD  target
961   0.099    -0.33            -0.94  257.566284   3.615669  0.440086     0.0
962   0.201     0.18            -1.06  257.355957   2.098551  0.385240     0.0
963   0.201     0.18            -1.06  268.123413  11.839519  0.620212     0.0
964   0.201     0.18            -1.06  263.010590   6.840503  0.447818     0.0
965   0.201     0.18            -1.06  259.632812   5.360566  0.302661     0.0

### Panel 1 last 5 rows ###
     OCI_AO  OCI_NAO  OCI_NINA34_ANOM    T2M_MEAN         TP       VPD  target
961   0.099    -0.33            -0.94  257.308868   3.405934  0.450177     0.0
962   0.201     0.18            -1.06  257.170746   2.299670  0.366090     0.0
963   0.201     0.18            -1.06  268.234100  11.543229  0.619684     0.0
964   0.201     0.18            -1.06  262.839966   7.076499  0.403347     0.0
965   0.201     0.18            -1.06  259.227203   5.587930  0.267106   

AttributeError: 'PCMCI' object has no attribute 'pcmci_parallel'

## Causal GNN

In [ ]:
model = CausalGNN(
    adj_matrix=norm_matrix,
    num_nodes=config['gnn_nodes'],
    hidden_dim=config['hidden_dim']
).to(device)

model_stats = summary(model)